# TP1- Revision sobre los SGBD relacionales

Para revisar algunos conceptos de los SGBD relacionales (en particular el modelo relacional y el lenguaje SQL), vamos a crear una pequeña Base de Datos relacional desde un programa en Python y a partir de datos que se encuentran en varios archivos textos.

La base de datos que queremos construir está representada por el modelo Entidad-Asociación siguiente:

<img src="modeloEA-1.png">

## 1. Transformación del Modelo Entidad-Asociación hacia Modelo Relacional

Proponer un modelo relacional que corresponde al modelo Entidad-Asociación.

medios_de_prensas(pk_nombre, duenho, num_followers)

noticias(id, titulo, text, #pk_nombre, date)

## 2. Crear la estructura de la base de datos desde un programa Python

### 2.1 Conectarse a MySQL desde Python

In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
from __future__ import print_function
import mysql.connector
from mysql.connector import errorcode

conexion = mysql.connector.connect(user='root', password='root',host='localhost')
if(conexion):
    print("conectado")
else:
    print("No conectado")
cursor = conexion.cursor()

conectado


### 2.2 Crear las tablas

In [4]:
DATABASE = {}
DB_NAME = "TB1_medios"
DATABASE['medios_de_prensas'] = (
    "CREATE TABLE `medios_de_prensas` ("
    "  `name` VARCHAR(255) NOT NULL,"
    "  `duenho` VARCHAR(255)  NOT NULL,"
    "  `num_followers` INT NOT NULL,"
    "  PRIMARY KEY (`name`)"
    ") ENGINE=InnoDB")
DATABASE['noticia'] = (
    "CREATE TABLE `noticia` ("
    "  `id` INT NOT NULL AUTO_INCREMENt,"
    "  `titulo` VARCHAR(255) NOT NULL,"
    "  `text` TEXT NOT NULL,"
    "  `ref_prensa` VARCHAR(255) NOT NULL,"
    "  `date` VARCHAR(255) NOT NULL,"
    "   PRIMARY KEY (`id`)"
    ") ENGINE=InnoDB")

def create_database(cursor):
    try:
        cursor.execute(
            "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(DB_NAME))
    except mysql.connector.Error as err:
        print("No se pudo conectar la base de datos: {}".format(err))
        exit(1)

try:
    conexion.database = DB_NAME  
except mysql.connector.Error as err:
    if err.errno == errorcode.ER_BAD_DB_ERROR:
        create_database(cursor)
        conexion.database = DB_NAME
    else:
        print(err)

try:
    cursor.execute(DATABASE['medios_de_prensas'])
    cursor.execute(DATABASE['noticia'])
except mysql.connector.Error as err:
    if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
        print("Las tablas ya han sido creadas")
    else:
        print(err.msg)
else:
    print("Tablas creadas con exito")       


Las tablas ya han sido creadas


# 3. Insertar los datos en la base de datos

## 3.1 Leer los archivos textos

In [5]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
import csv

with open('medios.csv') as csvarchivo:
    entrada = csv.DictReader(csvarchivo)
    for elementos in entrada:
        print(elementos['CódigoMedio'], " \t " ,elementos['PropiedadLegal'], " \t ",elementos['FollowersFinAño'])

24HorasTVN  	  Estado Chileno  	  3286603
TVN  	  Estado Chileno  	  2910624
T13  	  Grupo Canal 13  	  2869408
CNNChile  	  Time Warner  	  2814705
biobio  	  BIO-BIO comunicaciones  	  2756031
Cooperativa  	  Compañía Chilena de Comunicaciones  	  2638045
canal13  	  Grupo Luksic  	  2089537
latercera  	  Grupo Copesa  	  1849215
Emol  	  El Mercurio S.A.P.  	  1805524
adnradiochile  	  Ibero Americana Radio Chile  	  1660965
thecliniccl  	  Ediciones y Publicaciones Bobby S.A.   	  1637257
elmostrador  	  La Plaza S.A.  	  1533717
chilevision   	  Time Warner  	  1264464
terraChile  	  Terra Networks Chile S.A.  	  1190512
Mega  	  Bethia  	  1016622
ahoranoticiasAN  	  Bethia  	  1012851
ciper  	  Fundacion CIPER- Grupo Copesa  	  902201
PublimetroChile  	  Metro International  	  876749
lacuarta  	  Grupo Copesa  	  763174
deportes13cl  	  Grupo Luksic  	  610516
La_Segunda  	  El Mercurio S.A.P.  	  563406
LaRedTV  	  Albavisión (MX)  	  531088
el_dinamo  	  Ediciones Giro País S

## 3.2 Leer los archivos textos y insertar los datos

In [8]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

with open('medios.csv') as csvarchivo:
    entrada = csv.DictReader(csvarchivo)
    for reg in entrada:
        name=reg['CódigoMedio']
        duenho=reg['PropiedadLegal']
        num_followers=reg['FollowersFinAño']
        cursor.execute("INSERT INTO medios_de_prensas (name, duenho, num_followers) VALUES (%s, %s, %s)",(name,duenho,num_followers))
    conexion.commit()

archivo = open("noticias.txt")

datos = []
for linea in archivo.readlines(): 
    datos.append(linea) 
for i in range (0,int((len(datos)/5))):
    fecha = datos[5*i]
    ref_prensa = datos[5*i+1]
    titulo = datos [5*i+2]
    text = datos [5*i+3]
    cursor.execute("INSERT INTO noticias (titulo, date, ref_prensa, text) VALUES (%s, %s, %s, %s)",(titulo, fecha, ref_prensa, text))
conexion.commit()
archivo.close() 




IntegrityError: 1062 (23000): Duplicate entry '24HorasTVN' for key 'PRIMARY'

# 4. Consultar la base de datos

Listar los medios de prensa en Chile ordenado según su número de followers en Twitter.

In [ ]:

consulta = ("SELECT name, num_followers FROM medios_de_prensas ORDER BY num_followers DESC")
cursor.execute(consulta)

for (dato) in cursor:
  print(dato)


In [ ]:
consulta = ("SELECT COUNT(*) FROM (SELECT COUNT(duenho)FROM medios_de_prensas GROUP BY duenho) AS numerito")
cursor.execute(consulta)

for (nombre) in cursor:
  print(nombre)

Contar cuantes medios de prensa posee cada dueño.

In [ ]:
consulta = ("SELECT COUNT(name) AS valorcito, duenho FROM medios_de_prensas GROUP BY duenho")
cursor.execute(consulta)

for (nombre) in cursor:
  print(nombre)

Mostrar los últimos 10 títulos de noticias del medio "La Tercera"

In [ ]:
consulta = ("SELECT titulo FROM noticia WHERE ref_de_prensa LIKE'La Tercera' LIMIT 10 ")
cursor.execute(consulta)

for (nombre) in cursor:
  print(nombre)

Mostrar los nombre de medios y los títulos en los cuales aparacen la palabra "Valdivia".

In [ ]:
consulta = ("SELECT titulo, ref_de_prensa FROM noticia WHERE titulo LIKE '%Valdivia%' OR texto LIKE '%Valdivia%'")
cursor.execute(consulta)

for (nombre) in cursor:
  print(nombre)


Por cada medio, mostrar el número de títulos en los cuales aparecen la palabra "Puerto Montt".

In [ ]:
consulta = ("SELECT ref_de_prensa, COUNT(titulo) FROM noticia WHERE titulo LIKE '%Puerto Montt%' GROUP BY ref_de_prensa")
cursor.execute(consulta)

for (nombre) in cursor:
  print(nombre)